In [ ]:
from numpy import *
from matplotlib.pyplot import *
import scipy

from optics_design_workbench import jupyter_utils

# Run simulations for various lens radii

In [ ]:
#simulationMode = 'true'
simulationMode = 'pseudo'
#simulationMode = 'fans'

In [ ]:
# prefer using this cell to open the FreeCAD Gui. This call is blocking
# until FreeCAD is closed again, which helps to avoid editing a FCStd 
# file from jupyter and the FreeCAD Gui in parallel.

#jupyter_utils.openFreecadGui()

In [ ]:
# prepare arrays/lists for parameters/results
radii = linspace(30, 50, 15)
results = []

# open FCStd Document as a context manager. The context manager makes sure
# the file is closed again with certainty. Keep in mind that if the file is
# open in another FreeCAD GUI, saving from the Gui will overwrite the file
# and may leads to inconsistent file versions being seen by different 
# worker processes.
with jupyter_utils.FreecadDocument() as f:

  # setup progress tracking
  jupyter_utils.setupProgressTracker(totalSimulations=len(radii))

  # try different radii
  for rad in radii:

    # update radius and placement of both spheres that form the lens
    f.Sphere001.Radius = rad
    f.Sphere001.Placement.Base = [0, 0, -rad/1.9]
    f.Sphere.Radius = rad
    f.Sphere.Placement.Base = [0, 0, rad/1.9]

    # run a simulation
    results.append( f.runSimulation(simulationMode) )


In [ ]:
# show list of the first ten results
results[:10]

In [ ]:
# print filesystem tree overview of first result file:
results[0].printTree()

# Create histograms and do Gauss fit for each histogram

In [ ]:
gauss = lambda X, amp, x0, sigma: amp*exp(-(X-x0)**2/sigma**2)

In [ ]:
sigmas, x0s, _Hs = [], [], []
for r in results:
  hits = r.loadHits('*')
  hist = hits.histogram(bins=(300, 301))

  #figure(figsize=(4,4))
  #pcolormesh(X, Y, H.T)
  #show()
  
  #figure(figsize=(4,4))
  H, X = hist.hist, hist.binX
  _H = H.sum(axis=1)
  #plot(X, _H)
  try:
    (amp, x0, sigma), _ = scipy.optimize.curve_fit(gauss, X, _H, p0=(max(_H), 0, 1))
  except Exception:
    amp, x0, sigma = nan, nan, nan
  #plot(X, gauss(X, amp, x0, sigma))
  #show()

  sigmas.append(sigma)
  x0s.append(x0)
  _Hs.append(_H)

In [ ]:
sigmas, x0s, _Hs = [array(A) for A in (sigmas, x0s, _Hs)]

In [ ]:
import matplotlib.colors as col
_Hs = array(_Hs)
pcolormesh(X[:-1], radii, _Hs, norm=col.Normalize(_Hs.min(), _Hs.max()/5, clip=True))
xlabel('X coordinate')
ylabel('lens radius')
savefig('lens-optimizer-sweep-result-cmap.jpg')

In [ ]:
plot(radii, sigmas)
plot(radii, abs(x0s))
xlabel('lens radius')
ylabel('sigma (blue) and |x0| (orange)')
semilogy()
savefig('lens-optimizer-sweep-result-sigma-vs-radii.jpg')